In [1]:
import numpy as np
import pandas
from sklearn.semi_supervised import LabelPropagation, LabelSpreading
from sklearn.model_selection import cross_val_score
import pickle

## Constants

In [2]:
file_path = "/nfs/lab/varpred/Results/biobank_imbal_and_peaks"
t1d_mat_imbalance_binary = file_path + '/T1D_mat_imbalance_binary_val.txt'
t1d_mat_peaks_binary = file_path + '/T1D_mat_peaks_binary_val.txt'
t1d_mat_imbalance_actual = file_path + '/T1D_mat_imbalance_actual_val.txt'
t1d_mat_peaks_actual = file_path + '/T1D_mat_peaks_actual_val.txt'
t2d_mat_imbalance_binary = file_path + '/T2D_mat_imbalance_binary_val.txt'
t2d_mat_peaks_binary = file_path + '/T2D_mat_peaks_binary_val.txt'
t2d_mat_imbalance_actual = file_path + '/T2D_mat_imbalance_actual_val.txt'
t2d_mat_peaks_actual = file_path + '/T2D_mat_peaks_actual_val.txt'
proba_threshold = 0.8


# Data Ingestion

In [4]:
t1d_imbalance_binary = pandas.read_csv(t1d_mat_imbalance_binary, sep=' ')
t1d_peaks_binary = pandas.read_csv(t1d_mat_peaks_binary, sep=' ')
t1d_imbalance_actual = pandas.read_csv(t1d_mat_imbalance_actual, sep=' ')
t1d_peaks_actual = pandas.read_csv(t1d_mat_peaks_actual, sep=' ')


In [5]:
# t1d_peaks_actual = t1d_peaks_actual.rename(columns={"endo": "endocrine"})
# t1d_peaks_binary = t1d_peaks_binary.rename(columns={"endo": "endocrine"})

cell_types = list(t1d_imbalance_actual.columns)[2:]
raw_data_frames = {
    "imbalance_binary": t1d_imbalance_binary,
    "peaks_binary": t1d_peaks_binary,
    "imbalance_actual": t1d_imbalance_actual,
    "peaks_actual": t1d_peaks_actual,
}

data = pandas.concat([raw_data_frames['imbalance_actual'], raw_data_frames['imbalance_binary'], raw_data_frames['peaks_actual'], raw_data_frames['peaks_binary']], axis=1)
labels = data['probability'].iloc[:,0]
data = data.drop(['variant_id', 'probability'], axis=1)

In [6]:
labels = t1d_imbalance_binary['probability']

# Categorical labelling
def label_prob(prob):
    if prob > proba_threshold:
        return 1
    elif prob < 1 - proba_threshold:
        return 0
    else:
        return -1

labels = labels.apply(lambda row: label_prob(row))

# Label Propagation

In [ ]:
label_prop_model = LabelPropagation()
label_prop_model.fit(data, labels)

In [ ]:
pickle.dump(label_prop_model, open("./models/label_prop_model", 'wb'))

# Cross Validation

In [ ]:
# Filtering out unlabelled data
known_data_bool = labels != -1
data.insert(len(data.columns), 'label', labels)
data.insert(len(data.columns), 'known', known_data_bool)
all_known_data = data[data.known == True]
known_data = all_known_data.drop(['label', 'known'], axis=1)
known_labels = all_known_data['label']

In [ ]:
cv_score = cross_val_score(label_prop_model, known_data, y=known_labels, verbose=1, n_jobs=4)

# Label Spreading

In [ ]:
label_spread_model = LabelSpreading(alpha=(1 - proba_threshold))
# label_spread_model.fit(data, labels)

# Experimenting